# Basics
This example works with RDD. The purpose of this is example is solely get to know the technology. You will probably not need RDD's in your project as we wil use Spark SQL which is more flexible to use.

In [1]:
import datetime
#Make sure pyspark is installed as a package of your project.
from pyspark import SparkConf, SparkContext
import EnvironmentSetup as es
es.setupEnvironment()


# Basic setup to be able to use a Spark Cluster

In [2]:

#1. Creating a configuration to add parameters.
conf = SparkConf().setAppName("firstJob").setMaster("local[*]").setIfMissing("spark.logLineage", "true")
#setMaster() is used to define on which cluster the code has to run.
#   If you submit a job on the cluster itself the Master will be already set. from
#   If you want to run the code in your development environment, you have to set the master to local[*]. Spark will initiate a local-cluster that uses the different threads of your CPU.
# IMPORTANT: Never use setMaster("local[*]") hard-coded in your final code. When running on a real environment, overwriting the default with local[*] means the job will just run locally on the master node and the cluster itself will not be used to run the job.

#2. Create a sparkcontext. The context is used to initiate a processing job on the cluster
sc =SparkContext.getOrCreate((conf))
sc.uiWebUrl #just returns the sparkUI string. Visit the link below to get insights in the spark jobs

'http://AKDGPORT11191.admin.kdg.be:4042'

## Running WordCount (the Hello World of distributed processing)

In [4]:
import datetime

# 1. Use the Spark Context to read from an input. This is the start point for the Spark excecution engine. Reading from a textFile returns a dataframe with a record for each line.
# TODO: Use the correct path notation (eg. file:///, gs://, hdfs://) and add this to the arguments of the job instead of hard-coded.
lines = sc.textFile("./FileStore/tables/shakespeare.txt")\
#2. The first transformation transforms the lines to words
words = lines.flatMap(lambda line: line.split(" "))
#3. The second transformations creates a keyvalue map: map to key= word, value= 1
wordKv = words.map(lambda word: (word, 1))
#4. In this transformations an aggregation by key is performed. Every word is counted. Because records with the same key ar spread over the cluster, a shuffle is needed (e.g. all records with key "romeo" will be sent to one node). The execution engine will start a new stage for this step because of the shuffle.

wordCounts = wordKv.reduceByKey(lambda a,b:a +b)
#5. This step is not necessary but will cache the result to memory.
# This is more efficient if the wordCounts dataframe is needed for more new dataframes to avoid that Spark recalculates everything from the beginning for each chain.
wordCounts.cache()
# 6. Save the counts to output. As this is an action, the Spark execution engine will only now decide that all the preceding transformations do need to be executed.
print(wordCounts.saveAsTextFile("./output/words" + datetime.datetime.now().strftime("%m%d%Y%H%M%S")))
print(wc2.saveAsTextFile("./output/words2" + datetime.datetime.now().strftime("%m%d%Y%H%M%S")))

None


> ## Task 1
> Go to the SparkUI Url provided in the output and try to answer following questions:
> 1. Run the job. How many stages are there and why?
> 2. What changes in the execution plan when you un/comment wordCounts.cache()?
> 3. What changes in the Storage-tab when you un/comment wordCounts.cache()?
> 4. Copy the last line in the code and change /words to /words2 in the copied line.
>    Run the job without caching.
> _You will notice the creation of two identical jobs. The engine creates a job for every "action" in the code. For every action it will kcreate a new excecution plan and all tranformations will be processed twice._
> 5. Enable "caching" again and run the code again. Go to the last job. What is changed?
> _By enabling caching, the first job will store the intermediate result. The second job will reuse this result, which will prevent processing the same steps twice. With caching, you can choose between processing and memory. When enough memory is available you can greatly increase the performance of the jobs._


In [5]:

# Stops the spark context so you can build a new one.
sc.stop()